In [ ]:
import concurrent.futures
import datetime
import os
import traceback
from pathlib import Path

import dateparser
import pandas as pd
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/Users/Felis.catus/Desktop/09-29-2021_kraken-769a9b6b_part1_ACC_LS_unlabeled.csv')

In [ ]:
file = '/Users/Felis.catus/Desktop/GitHub/Biodiversity-CatTracker2/CatTracker2.0-Database/cats/kraken-769a9b6b/10-03-2021/RAW/10-03-2021_kraken-769a9b6b_raw_ACC.csv.tbz2'
df = pd.read_csv(file, compression='bz2')
df.rename(columns={Path(file).stem: 'event'}, inplace=True)
df.drop([len(df) - 1], axis=0, inplace=True)
df

In [ ]:
date = dateparser.parse('10-03-2021')

In [ ]:
intervals = []
new_day = False
for x, y in zip(df['Time'], df['event']):
    if x == '00:00:01':
        new_day = False
    if new_day:
        continue
    if x == '00:00:00':
        new_day = True
        intervals.append((int(y), x))
idx = [x[0] for x in intervals]
intervals, idx

In [ ]:
df['Date'] = [date] * len(df)
df['Date'][:idx[0]] = date
df['Date'][idx[0]:idx[1]] = date + datetime.timedelta(days=1)
df['Date'][idx[1]:idx[2]] = date + datetime.timedelta(days=2)
df['Date'][idx[2]:] = date + datetime.timedelta(days=3)
df['Date'] = [str(x.date()) for x in df['Date']]
len(df['Date']) == len(df)

In [ ]:
df

In [ ]:
df = df[['event', 'tag.local.identifier', 'Date'] + [x for x in df.columns
         if x not in ['Date', 'event', 'tag.local.identifier']]]
df

In [ ]:
df['timestamp'] = df['Date'] + ' ' + df['Time']
left_cols = ['event', 'tag.local.identifier', 'timestamp']
df = df[left_cols + [x for x in df.columns if x not in left_cols]]
df

In [ ]:
bad_fmt = []
for x in df['timestamp']:
    try:
        datetime.datetime.strptime(x, '%m-%d-%Y %H:%M:%S')
    except Exception as e:
        bad_str = traceback.format_exc().split('\n')[-2].split('\'')[1]
        bad_fmt.append(list(df.loc[(df['timestamp'] == bad_str)]['event']))

if bad_fmt:
    assert all([isinstance(x, list) for x in bad_fmt])
    bad_idx = [x - 1 for x in set([i for l in bad_fmt for i in l])]
    assert len(set(list(df.iloc[bad_idx]['timestamp']))) == 1

In [ ]:
# if bad_fmt:
#     df.iloc[bad_idx]

In [ ]:
if bad_fmt:
    date_before = df['timestamp'][bad_idx[0]-1]
    date_before_len = len(df.loc[df['timestamp'] == date_before])
    date_before_len, date_before

In [ ]:
if bad_fmt:
    date_after = df['timestamp'][bad_idx[-1]+1]
    date_after_len = len(df.loc[df['timestamp'] == date_after])
    date_after_len, date_after

In [ ]:
if bad_fmt:
    if date_before_len == 16:
        df.replace(list(df.iloc[bad_idx]['timestamp'])[0], date_after, inplace=True)
        time_after = dateparser.parse(date_after).time().strftime('%H:%M:%S')
        df.replace(list(df.iloc[bad_idx]['Time'])[0], time_after, inplace=True)

    elif date_after_len == 16:
        df.replace(list(df.iloc[bad_idx]['timestamp'])[0], date_before, inplace=True)
        time_before = dateparser.parse(date_before).time().strftime('%H:%M:%S')
        df.replace(list(df.iloc[bad_idx]['Time'])[0], time_before, inplace=True)

    else:
        raise Exception('Neither seconds have 16 data points!')

In [ ]:
if bad_fmt:
    df.iloc[bad_idx]

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%m-%d-%Y %H:%M:%S')

df = df.astype(
    {
        'event': 'int32',
        'tag.local.identifier': 'int32',
        'timestamp': 'datetime64[ns]',
        'Accx': 'float32',
        'Accy': 'float32',
        'Accz': 'float32',
        'rAccx': 'int32',
        'rAccy': 'int32',
        'rAccz': 'int32',
    }
)
df.dtypes

In [ ]:
new_file_path = f'{Path(file).parent}/{Path(Path(file).stem).stem}_with_date.csv'
df.to_csv(new_file_path, index=False)
print('Now compress with Keka...')